## Invoke SageMaker Enpoint from outside of AWS environment using SageMaker SDK

Model used: XGBoost Bike Rental Prediction Trained in the XGBoost Lectures  
  
This example uses the IAM user: ml_user_predict. The user was setup in the housekeeping lecture of the course.  

Refer to the lecture: Configure IAM Users, Setup Command Line Interface (CLI)

Ensure xgboost-biketrain-v1 Endpoint is deployed before running this example  
  
To create an endpoint using SageMaker Console:  
1. Select "Models" under "Inference" in navigation pane
2. Search for model using this prefix: xgboost-biketrain-v1
3. Select the latest model and choose create endpoint
4. Specify endpoint name as: xgboost-biketrain-v1
5. Create a new endpoint configuration
6. Create a new endpoint
7. After this lab is completed, delete the endpoint to avoid unnecessary charges

In [1]:
# Install SageMaker 2.x version.
#!pip install --upgrade sagemaker

In [2]:
import boto3
import sagemaker
import math
import dateutil
import re
import numpy as np

# SDK 2 serializers and deserializers
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

In [3]:
# Establish a session with AWS
# Specify credentials and region to be used for this session.
# We will use a ml_user_predict credentials that has limited privileges

#boto_session = boto3.Session(profile_name='ml_user_predict',region_name='us-east-1')

boto_session = boto3.Session(profile_name='ml_user_predict', region_name='us-east-1')


ProfileNotFound: The config profile (ml_user_predict) could not be found

In [4]:
type(boto_session)

boto3.session.Session

In [5]:
sess = sagemaker.Session(boto_session=boto_session)

In [6]:
# Create a predictor and point to an existing endpoint

# Get Predictor using SageMaker SDK
# Specify Your Endpoint Name
endpoint_name = 'kmeans-2022-11-22-09-34-16-400'

predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name,
                                                 sagemaker_session=sess)

In [7]:
# We are sending data for inference in CSV format
predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()
#predictor.deserializer = json_deserializer

# Invoke endpoint to predict for multiple observations at once

In [23]:
observation_1 = np.array([[-0.7319894 , -0.24943449, -0.42489973, -0.356934  , -0.3490788 ,
       -0.46678555, -0.80649036, -0.6786608 , -0.7073132 , -0.6753489 ,
       -0.4760698 , -0.5113332 , -0.96043336, -0.52897877, -0.31096753,
       -0.52555096,  0.36067954]])

In [24]:
observation_2 = np.array([[ 0.78696084,  0.13432467, -0.4695519 , -0.356934  , -0.45457622,
        2.6056044 , -1.221758  , -0.6786608 , -0.91699517,  0.57396275,
        0.11007385, -0.59179586,  0.68863904,  0.81864214,  0.08931021,
        0.2342269 ,  0.36067954]])

In [31]:
observation_3 = np.array([[ 0.44713512,  0.5180838 , -0.10766823,  0.10888851, -0.45457622,
       -0.46678555,  1.2698432 ,  2.6734512 , -0.91699517, -0.6753489 ,
       -0.4760698 , -0.10902011,  0.8260617 , -0.38380474, -0.10166318,
       -0.52555096,  0.36067954]])

In [32]:
# Let's invoke predictions
result = predictor.predict(observation_1)
result

{'predictions': [{'closest_cluster': 2.0,
   'distance_to_cluster': 1.2890762090682983}]}

In [33]:
labels = [pred['closest_cluster'] for pred in result['predictions']]
labels

[2.0]

# Invoke endpoint to predict for multiple observations at once

In [36]:
multiple_observations = np.concatenate((observation_1, observation_2, observation_3))
multiple_observations

array([[-0.7319894 , -0.24943449, -0.42489973, -0.356934  , -0.3490788 ,
        -0.46678555, -0.80649036, -0.6786608 , -0.7073132 , -0.6753489 ,
        -0.4760698 , -0.5113332 , -0.96043336, -0.52897877, -0.31096753,
        -0.52555096,  0.36067954],
       [ 0.78696084,  0.13432467, -0.4695519 , -0.356934  , -0.45457622,
         2.6056044 , -1.221758  , -0.6786608 , -0.91699517,  0.57396275,
         0.11007385, -0.59179586,  0.68863904,  0.81864214,  0.08931021,
         0.2342269 ,  0.36067954],
       [ 0.44713512,  0.5180838 , -0.10766823,  0.10888851, -0.45457622,
        -0.46678555,  1.2698432 ,  2.6734512 , -0.91699517, -0.6753489 ,
        -0.4760698 , -0.10902011,  0.8260617 , -0.38380474, -0.10166318,
        -0.52555096,  0.36067954]])

In [37]:
# Let's invoke predictions
result = predictor.predict(multiple_observations)
result

{'predictions': [{'closest_cluster': 2.0,
   'distance_to_cluster': 1.2890762090682983},
  {'closest_cluster': 0.0, 'distance_to_cluster': 2.3254904747009277},
  {'closest_cluster': 6.0, 'distance_to_cluster': 2.563176393508911}]}

In [39]:
labels = [int(pred['closest_cluster']) for pred in result['predictions']]
labels

[2, 0, 6]